In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import tkinter as tk
from tkinter import messagebox
import joblib

from tkinter import ttk
import emoji
import threading
from PIL import Image, ImageTk   
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer


data = pd.read_csv("Dataset.csv")


data=data[['rating','title','text']]


data=data.dropna()


data['rating'].unique()


data.shape



data['rating']=data['rating'].astype(float)


# Mapping function to convert ratings to labels
def map_to_label(rating):
    if rating > 4:
        return 0
    elif rating <=4 and rating >=3:
        return 1
    else:
        return 2
    


# Apply the mapping function to create a new 'sentiment' column
data['sentiment'] = data['rating'].apply(map_to_label)


data.head()


data.sentiment.unique()


# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


# Preprocess the text data
vectorizer = CountVectorizer(stop_words='english')



X_train = vectorizer.fit_transform(train_data['text'])
y_train = train_data['sentiment']

X_test = vectorizer.transform(test_data['text'])
y_test = test_data['sentiment']


# Train a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)


# Paths of images that express the result of processing
positive_emoji_path = "Emojis/positive.png"
neutral_emoji_path = "Emojis/neutral.png"
negative_emoji_path = "Emojis/negative.png"

# Initialize the sentiment label as None
sentiment_label = None




def check_review():

    # Declare sentiment_label as a global variable
    global sentiment_label  

    new_review = review_entry.get()  # Get the review as a string
    new_review_vectorized = vectorizer.transform([new_review])  # Transform it into a list
    prediction = clf.predict(new_review_vectorized)[0]  # Get the prediction for the single review

    # Print the prediction for debugging
    print(f"Prediction: {prediction}")

    # Remove the old sentiment label if it exists
    if sentiment_label is not None:
        sentiment_label.destroy()

    sentiment_emoji = None

    if prediction == 0:
        sentiment_emoji = Image.open(positive_emoji_path)
    elif prediction == 1:
        sentiment_emoji = Image.open(neutral_emoji_path)
    else:
        sentiment_emoji = Image.open(negative_emoji_path)

    sentiment_emoji = sentiment_emoji.resize((200, 200))  

    # Displaying the image in a label
    sentiment_image = ImageTk.PhotoImage(sentiment_emoji)
    sentiment_label = tk.Label(root, image=sentiment_image)
    sentiment_label.image = sentiment_image  # Keeping a reference to avoid garbage collection
    sentiment_label.pack(pady=20)



# Clearing the text in the review entry 
def clear_review():
    review_entry.delete(0, tk.END) 






#-----------------------GUI--------------------------------------    

# Main window widgets
root = tk.Tk()
root.title("Review Checker")
window_width = 800
window_height = 600
root.geometry(f"{window_width}x{window_height}")
root.configure(bg="#3b4151")



# Set the window size and center it on the screen
window_width = 800
window_height = 600
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{x}+{y}")


# Set the border width and relief (border appearance)
border_width = 5  
border_relief = "ridge"  
root.configure(borderwidth=border_width, relief=border_relief)


label = tk.Label(root, text="Enter a review", font=("Arial", 16, "bold"), fg="#3b4151", bg="#ccc")
label.pack(pady=20)


review_entry = tk.Entry(root, width=50, font=("Arial", 12), bd=2, relief="solid",  bg="#ccc")
review_entry.pack(pady=10, padx=10, ipady=5, ipadx=5)


check_button = tk.Button(root, text="Check Review", command=check_review, font=("Arial", 14, "bold"), bg="green", fg="white", bd=3, relief="raised")
check_button.pack(pady=20)


clear_button = tk.Button(root, text="Clear",       command=clear_review, font=("Arial", 14, "bold"), bg="red", fg="white", bd=3, relief="raised")
clear_button.pack(pady=10)




root.mainloop()

